In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import os
import datajoint as dj
dj.config['database.host'] = os.environ['DJ_HOST']
dj.config['database.user'] = os.environ['DJ_USER']
dj.config['database.password'] = os.environ['DJ_PASS']
dj.config['enable_python_native_blobs'] = True
dj.config['schema_name'] = "anix_nnfabrik_bias_transfer1"
# dj.config['schema_name'] = "anix_nnfabrik_bias_transfer_test2"

import matplotlib.pyplot as plt
import numpy as np
from bias_transfer.dataset import dataset_loader
from bias_transfer.trainer import trainer
from bias_transfer.models import resnet_builder
from bias_transfer.analysis import *
from bias_transfer.main import *


from bias_transfer.tables.trained_model import *
from bias_transfer.tables.trained_transfer_model import *
from bias_transfer.tables.evaluated_model import *
import nnfabrik as nnf
# nnf.config['repos'] = ['/notebooks/nnfabrik']
# from nnfabrik.main import *
schema

Connecting anix@sinzlab.chlkmukhxp6i.eu-central-1.rds.amazonaws.com:3306


Schema `anix_nnfabrik_bias_transfer1`

In [2]:
import imageio

In [55]:
from bias_transfer.experiments.representation_analysis import experiments
experiment_key = "clean"
dataset_cls = "CIFAR10"
idx = True
if experiment_key == "clean":
    exp = experiments[Description(name=dataset_cls + ": Clean", seed=42)]
if experiment_key == "noisy":
    exp = experiments[Description(name=dataset_cls + ": Noise Augmented", seed=42)]
if experiment_key == "rep_matching":
    exp = experiments[Description(name=dataset_cls + ": Noise Augmented + Repr. Matching", seed=42)]
if experiment_key == "adv_regression":
    exp = experiments[Description(name=dataset_cls + ": Noise Augmented + Noise Adv Regession", seed=42)]

In [56]:
val_analyser = RepresentationAnalyser(experiment=exp, table=TrainedModel(), dataset="val",
                                      plot_style="lightpaper")

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Model with 23520842 parameters.


In [57]:
path = "./bias_transfer/bias_transfer/analysis/"

In [58]:
filenames = []
filenames.append(os.path.join(path,val_analyser._get_name(method="corr", mode="clean", noise_level=0.0)+"_plot.png"))
for i in range(1, 21):
    noise_level = 0.05 * i
    if not idx:
        filenames.append(os.path.join(path,val_analyser._get_name(method="corr", mode="noisy", noise_level=noise_level)+"_plot.png"))
    else:
        filenames.append(os.path.join(path,val_analyser._get_name(method="corr_given_idx", mode="noisy", noise_level=noise_level)+"_plot.png"))

In [59]:
images = []
for filename in filenames:
    images.append(imageio.imread(filename))
imageio.mimsave('./{}_{}.gif'.format(experiment_key, "given_idx" if idx else ""), images, format='GIF', duration=2)